In [45]:
import pandas as pd, numpy as np, json, os
import matplotlib.pyplot as plt
%matplotlib inline

In [46]:
plt.style.use('seaborn-whitegrid')

In [47]:
#!pip install d3IpyPlus
from d3IpyPlus import ScatterPlot, LinePlot, BarPlot, StackedArea

In [48]:
#!pip install pyecharts

In [49]:
m=json.loads(open('ignore/member_timelines.json','r').read())
members=json.loads(open('ignore/member_simple.json','r').read())

In [50]:
d=pd.DataFrame(members['deaths'],index=['v']).T

In [51]:
d['v']=' †'

In [52]:
s=pd.DataFrame(members['s2'],index=['s']).T
s=s.join(d)
s['v']=s['v'].fillna('')
s['s']=s['s']+s['v']
s=s[['s']]
s['w']=s['s'].str.split(' ').str[0].str[0]+s['s'].str.split(' ').str[-1].str[0]

In [54]:
szd_color='#E91C62'

In [55]:
pcolors=json.loads(open('json/pcolors.json','r').read())

In [56]:
g=pd.DataFrame(members['megye'],index=['Megye ']).T
g.head()

,Megye
Alexandrescu Emil Lucian | 1937-11-19,🇷🇴 Iași
Dobritoiu Teodoru | 1954-06-14,🇷🇴 Galac
Stef Mircea-Ioan | 1930-11-07,🇷🇴 Iași
Barbat Vasile | 1937-10-06,🇷🇴 Temes
Dumitrescu Alexandru | Ismeretlen,🇷🇴 Buzău


In [63]:
data={}
attention={}
for k,i in enumerate(m):
    if k%1000==0: print(k/len(m)*100,'%')
    if i['Típus']=='Párt':
        if i['Csoportok'] not in data:data[i['Csoportok']]={'start':pd.to_datetime('2030'),'end':pd.to_datetime('1980')}
        start=pd.to_datetime(i['start'])
        end=pd.to_datetime(i['end'])
        data[i['Csoportok']]['start']=min(data[i['Csoportok']]['start'],start)
        data[i['Csoportok']]['end']=max(data[i['Csoportok']]['end'],end)

0.0 %
3.3029462280354074 %
6.605892456070815 %
9.908838684106223 %
13.21178491214163 %
16.514731140177037 %
19.817677368212447 %
23.120623596247853 %
26.42356982428326 %
29.72651605231867 %
33.029462280354075 %
36.332408508389484 %
39.635354736424894 %
42.938300964460296 %
46.241247192495706 %
49.544193420531116 %
52.84713964856652 %
56.15008587660193 %
59.45303210463734 %
62.75597833267275 %
66.05892456070815 %
69.36187078874356 %
72.66481701677897 %
75.96776324481438 %
79.27070947284979 %
82.57365570088518 %
85.87660192892059 %
89.179548156956 %
92.48249438499141 %
95.78544061302682 %
99.08838684106223 %


In [216]:
part_timelines=pd.DataFrame(data).T
part_timelines.index.name='id'

In [217]:
part_timelines['start']=part_timelines['start'].astype(str).str[:10]
part_timelines['end']=part_timelines['end'].astype(str).str[:10]

In [218]:
p=part_timelines.reset_index()[['id']]
p['Color']=p['id'].str.split(' ').str[1].str.strip()
p['Part']=p['id'].str.split('-').str[0].str.strip()

In [219]:
part_timelines=part_timelines.join(p.set_index('id').drop_duplicates())

In [220]:
logos={'🕊️':'ALDE', '🌹':'PSD', '📐':'PNL', '🔶':'PD', '⚛️':'PP-DD', '👤':'Független', '⚪️':'FSN',
       '✳️':'PNTCD', '🔱':'PRM', '🌷':'RMDSZ', '🍀':'ECO', '🍏':'PMP',
       '🔷':'USR', '🦅':'UNPR', '🔘':'Más'}

In [221]:
part_timelines['logo']=part_timelines['Part'].str.split(' ').str[0]

In [222]:
part_timelines['group']=[logos[i] for i in part_timelines['logo'].values]
part_timelines['label']=[i+' '+logos[i] for i in part_timelines['logo'].values]

In [223]:
part_timelines=part_timelines.join(pd.DataFrame(pcolors,index=['cvalue']).T,on='group').fillna('grey')

In [225]:
part_timelines=list(part_timelines.reset_index().T.to_dict().values())

In [226]:
open('json/part_timelines.json','w').write(json.dumps(part_timelines))

9874

In [227]:
part_timelines[3]

{'id': '🕊️ PUNR - Partidul Unităţii Naţionale Române',
 'end': '2000-11-30',
 'start': '1990-06-19',
 'Color': 'PUNR',
 'Part': '🕊️ PUNR',
 'logo': '🕊️',
 'group': 'ALDE',
 'label': '🕊️ ALDE',
 'cvalue': '#046cab'}

Resolve for non-activity


In [264]:
data={}
attention={}
for k,i in enumerate(m):
    if k%1000==0: print(k/len(m)*100,'%')
    if i['Típus']=='Párt':
        if i['Csoportok'] not in data:data[i['Csoportok']]=[]
        for t in pd.date_range(i['start'],i['end']):
            st=str(t)[:10]
            data[i['Csoportok']].append(st)

0.0 %
3.3029462280354074 %
6.605892456070815 %
9.908838684106223 %
13.21178491214163 %
16.514731140177037 %
19.817677368212447 %
23.120623596247853 %
26.42356982428326 %
29.72651605231867 %
33.029462280354075 %
36.332408508389484 %
39.635354736424894 %
42.938300964460296 %
46.241247192495706 %
49.544193420531116 %
52.84713964856652 %
56.15008587660193 %
59.45303210463734 %
62.75597833267275 %
66.05892456070815 %
69.36187078874356 %
72.66481701677897 %
75.96776324481438 %
79.27070947284979 %
82.57365570088518 %
85.87660192892059 %
89.179548156956 %
92.48249438499141 %
95.78544061302682 %
99.08838684106223 %


In [273]:
dt=30
for d in data:
    print(d)
    dates=np.sort(data[d])
    for i in range(1,len(dates)):
        if (pd.to_datetime(dates[i])-pd.to_datetime(dates[i-1])).days>dt:
            print(d,i)

🕊️ PC - Partidul Conservator


KeyboardInterrupt: 

In [293]:
a=pd.DataFrame(pd.to_datetime(dates[1:])).drop_duplicates().reset_index(drop=True)

In [294]:
b=pd.DataFrame(pd.to_datetime(dates[:-1])).drop_duplicates().reset_index(drop=True)

In [300]:
c=b-a
c['dt']=c[0].dt.days

In [301]:
c[c['dt']>dt]

,0,dt


In [257]:
[pd.to_datetime(values[j])-pd.to_datetime(values[j-1]) for j in range(1,len(values)-1)]

[end     -957 days
 start      0 days
 Name: 🕊️ PC - Partidul Conservator, dtype: timedelta64[ns], end     123 days
 start     0 days
 Name: 🕊️ PC - Partidul Conservator, dtype: timedelta64[ns], end     242 days
 start     0 days
 Name: 🕊️ PC - Partidul Conservator, dtype: timedelta64[ns], end     489 days
 start     0 days
 Name: 🕊️ PC - Partidul Conservator, dtype: timedelta64[ns], end     -366 days
 start      0 days
 Name: 🕊️ PC - Partidul Conservator, dtype: timedelta64[ns], end     -92 days
 start     0 days
 Name: 🕊️ PC - Partidul Conservator, dtype: timedelta64[ns], end     -61 days
 start     0 days
 Name: 🕊️ PC - Partidul Conservator, dtype: timedelta64[ns], end     -182 days
 start      0 days
 Name: 🕊️ PC - Partidul Conservator, dtype: timedelta64[ns], end     182 days
 start   409 days
 Name: 🕊️ PC - Partidul Conservator, dtype: timedelta64[ns], end     183 days
 start     0 days
 Name: 🕊️ PC - Partidul Conservator, dtype: timedelta64[ns], end     439 days
 start   273 day